# Summarizing meeting notes

Welcome to Cohere! In this notebook, you'll learn how to:
* Use Cohere's Command-R model to summarize meeting transcripts.
* Modify your prompt to include specific formatting instructions, especially if the model output is used in downstream applications.
* Use Command-R with LangChain for summarization.

### Table of contents:
1. [Setup](#setup)
2. [Load test data](#load)
3. [Simple summarization](#simple)
4. [More advanced formatting](#more)
5. [LangChain summarization](#langchain)
6. [Wrap up](#wrap)

<a id="setup"></a>
<a name="setup"></a>
## Setup

You'll need a Cohere API key to run this notebook. If you don't have a key, head to https://cohere.com/ to generate your key.

In [7]:
%%capture
!pip install cohere datasets tokenizers langchain langchain-cohere

In [8]:
import re
from string import Template
from typing import Optional

import cohere
from getpass import getpass
from datasets import load_dataset

# Set up Cohere client
co_api_key = getpass("Enter your Cohere API key: ")
co_model = "command-r"
co = cohere.Client(api_key=co_api_key)

In [9]:
# We'll also defining some util functions for later

def pprint(s: Optional[str] = None, maxchars: int = 100):
  """
  Wrap long text into lines of at most `maxchars` (preserves linebreaks occurring in text)
  """
  if not s:
    print()
  else:
    print("\n".join(line.strip() for line in re.findall(rf".{{1,{maxchars}}}(?:\s+|$)", s)))

<a id="load"></a>
<a name="load"></a>
## Load test data

Let's load a meeting transcript to see Command in action!

* If you have your own transcript, you can load it to Colab using your favorite method.
* If you don't, we'll use a sample from the [QMSum dataset](https://github.com/Yale-LILY/QMSum). QMSum contains cleaned meeting transcripts with [diarised speakers](https://en.wikipedia.org/wiki/Speaker_diarisation).
* We'll see later that the recipe shared herein isn't limited to meeting notes transcript, but extends to any data with diarised speakers!

In [10]:
# If you have your own transcript you would like to test Command on, load it here
# transcript = ...

# Otherwise, we'll use QMSum
# Note this will download the QMSum dataset to your instance
qmsum = load_dataset("MocktaiLEngineer/qmsum-processed")
# Pick any one transcript
transcripts = qmsum["validation"]["meeting_transcript"]
transcript = transcripts[0]
pprint(transcript[:2000])

PhD D: Sure because I I need a lot of time to to put the label or to do that
Professor E: I mean we we know that there s noise There s there s continual noise from fans and so
forth and there is more impulsive noise from taps and so forth and and something in between with
paper rustling We know that all that s there and it s a g worthwhile thing to study but obviously it
takes a lot of time to mark all of these things Whereas th i I would think that you we can study
more or less as a distinct phenomenon the overlapping of people talking So Then you can get the Cuz
you need If it s three hundred i i it sounds like you probably only have fifty or sixty or seventy
events right now that are really And and you need to have a lot more than that to have any kind of
even visual sense of of what s going on much less any kind of reasonable statistics
PhD C: Now why do you need to mark speaker overlap by hand if you can infer it from the relative
energy in the
Grad G: Well that s That s what I wa

Let's also check how many tokens that transcript corresponds to.

In [13]:
# Measure the number of tokens
num_tokens = len(co.tokenize(text=transcript, model=co_model).tokens)
pprint(f"Number of tokens: {num_tokens}")

Number of tokens: 1152


<a id="simple"></a>
<a name="simple"></a>
## Simple summarization

First, we'll show an example of a simple summarization task. The prompt template we are using is shown below, and we can obtain the completion using a one line call to the Cohere API.

In [14]:
prompt_template = Template("""## meeting transcript
$transcript

## instructions
Generate a summary of the above meeting transcript. \
Don't include preambles, postambles or explanations. \
""")
prompt = prompt_template.substitute(transcript=transcript)

# Use the Cohere API to get the response
resp = co.chat(message=prompt, model=co_model, temperature=0.2).text
pprint(resp)

The team discusses the time-consuming process of identifying instances of speaker overlap in audio
data. PhD C suggests using close-talking microphones and automated methods to establish ground truth
data, which Grad G is working on. This data would help develop a more efficient system than manual
marking.


We applied a few best practices in constructing this prompt:

1. We include a preamble imbuing the model with a persona.
2. We use Markdown-style headers (i.e. with `##`) to delineate the preamble, the text-to-summarise, the instructions, and the output

For some other prompting best practices and tips/tricks, see our [prompting guide](https://docs.cohere.com/docs/crafting-effective-prompts)!

<a id="more"></a>
<a name="more"></a>
## More advanced formatting

That worked well, but what if you want to have more control over the summary? What if you wanted a bulleted summary and wanted exactly 3 bullets in a json structure?

Let's try adding in some more specific formatting instructions.


In [15]:
prompt_template = Template("""## meeting transcript
$transcript

## instructions
Generate a summary of the above meeting transcript in 3 bullets. \
Use the following JSON format:
{
    "summary_bullets": [
        "<bullet 1>",
        "<bullet 2>",
        ...
    ]
}
Don't include preambles, postambles or explanations.\
""")
prompt = prompt_template.substitute(transcript=transcript)

# Use the Cohere API to get the response
resp = co.chat(message=prompt, model=co_model, temperature=0.2).text
print(resp)

```json
{
    "summary_bullets": [
        "There are many ways to study and mark speech overlap in a busy environment, but it's a time-consuming task.",
        "A threshold-based program that uses close-talking mics to infer speech overlap could speed up the process. The program looks for runs after applying a median filter and appears to work.",
        "Having a ground truth to compare the automated results against is important, and a human review of the data is still necessary."
    ]
}
```


That worked well! Asking for a structured response like a JSON object is a great approach for use cases where you will need to parse or post-process the output for use in downstream applications. To ensure the best possible completion, I provided a template of the expected JSON that the model filled out.

Let's also try asking for a short 1 sentence summary.

In [16]:
prompt_template = Template("""## meeting transcript
$transcript

## instructions
Generate a summary of the above meeting transcript in 1 concise sentence. Make sure the sentence is extremely short.\
Don't include preambles, postambles or explanations.\
""")
prompt = prompt_template.substitute(transcript=transcript)

# Use the Cohere API to get the response
resp = co.chat(message=prompt, model=co_model, temperature=0.2).text
pprint(resp)

The team discusses using various methods, including close-talking microphones and automated volume
thresholding, to accurately identify instances of speaker overlap in recorded meetings as part of a
research project.


<a id="langchain"></a>
<a name="langchain"></a>
## LangChain summarization
It's also very easy to use Command-R with LangChain for summarization.

In [17]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_cohere import ChatCohere
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate


llm = ChatCohere(
    cohere_api_key=co_api_key,
    model="command-r",
    temperature=0.2,
  )

# Define prompt
prompt_template = """## meeting transcript
{transcript}

## instructions
Generate a summary of the above meeting transcript. \
Don't include preambles, postambles or explanations."""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="transcript", verbose=True)

# Load documents
docs = [Document(page_content=transcript)]

pprint(stuff_chain.run(docs))

/Users/boyu/miniconda3/envs/test/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/boyu/miniconda3/envs/test/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...

> Finished chain.
The team discusses the time-consuming process of identifying instances of speaker overlap in audio
data. PhD C suggests using close-talking microphones and automated methods to establish ground truth
data, which Grad G is working on. This data would help develop a more efficient system than manual
marking.


Command-R has a context window of 128K tokens, so you can use very long documents, or multiple documents, all in a single API call.



<a id="wrap"></a>
<a name="wrap"></a>
## Wrap up

Those were some simple examples of how you can use Cohere's Command-R model for summarization.

For more advanced summarization objectives and to see some other cool things you can do with the Command-R model, see [recipes for better meeting notes summaries](https://github.com/cohere-ai/notebooks/blob/main/notebooks/guides/Recipes_for_better_meeting_notes_summary.ipynb).

In cases where you can't fit the input text into the context window, see our [cookbook for long document strategies](https://github.com/cohere-ai/notebooks/blob/main/notebooks/guides/Long_form_General_Strategies.ipynb).

For more information, you can reach out to us at summarize@cohere.com!